# Modelagem: montagem de elenco para times de futebol

## Importando bibliotecas e carregando dados

In [15]:
import pandas as pd
import numpy as np
import time

In [16]:
player_data = pd.read_csv("player-data-full.csv")

/var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/ipykernel_43436/484434192.py:1: DtypeWarning: Columns (15,16,34,35,36,37,39,73) have mixed types. Specify dtype option on import or set low_memory=False.
  player_data = pd.read_csv("player-data-full.csv")


In [17]:
player_data.head(3)

,player_id,version,name,full_name,description,image,height_cm,weight_kg,dob,positions,...,composure,defensive_awareness,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,play_styles
0,239085,240033,Erling Haaland,Erling Braut Haaland,"Erling Haaland (Erling Braut Haaland, born 21 ...",https://cdn.sofifa.net/players/239/085/24_120.png,195,94,2000-07-21,ST,...,87,38,47,29,7,14,13,11,7.0,"Acrobatic +,Power Header,Quick Step"
1,231747,240033,Kylian Mbappé,Kylian Mbappé Lottin,"Kylian Mbappé (Kylian Mbappé Lottin, born 20 D...",https://cdn.sofifa.net/players/231/747/24_120.png,182,75,1998-12-20,"ST,LW",...,88,26,34,32,13,5,7,11,6.0,"Quick Step +,Finesse Shot,Rapid,Flair,Trivela,..."
2,192985,240033,Kevin De Bruyne,Kevin De Bruyne,Kevin De Bruyne (born 28 June 1991) is a Belgi...,https://cdn.sofifa.net/players/192/985/24_120.png,181,75,1991-06-28,"CM,CAM",...,88,66,70,53,15,13,5,10,13.0,"Incisive Pass +,Dead Ball,Pinged Pass,Long Bal..."


In [18]:
player_data.columns

Index(['player_id', 'version', 'name', 'full_name', 'description', 'image',
       'height_cm', 'weight_kg', 'dob', 'positions', 'overall_rating',
       'potential', 'value', 'wage', 'preferred_foot', 'weak_foot',
       'skill_moves', 'international_reputation', 'work_rate', 'body_type',
       'real_face', 'release_clause', 'specialities', 'club_id', 'club_name',
       'club_league_id', 'club_league_name', 'club_logo', 'club_rating',
       'club_position', 'club_kit_number', 'club_joined',
       'club_contract_valid_until', 'country_id', 'country_name',
       'country_league_id', 'country_league_name', 'country_flag',
       'country_rating', 'country_position', 'country_kit_number', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'fk_accuracy', 'long_passing', 'ball_control',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
 

## Filtrando as colunas importantes

In [19]:
cols = ["name", "dob", "country_name", "positions", "overall_rating", "potential", "value", "wage",
        "club_name", "club_league_name", "country_name", "acceleration", "agility", "strength", "stamina"]

df = player_data[cols].copy()

df.head(1)

,name,dob,country_name,positions,overall_rating,potential,value,wage,club_name,club_league_name,country_name,acceleration,agility,strength,stamina
0,Erling Haaland,2000-07-21,Norway,ST,91,94,€185M,€340K,Manchester City,Premier League,Norway,82,78,93,76


Agora, seria interessante fazer alguns tratamentos:

- transformar a data de nascimento (`dob`) em idade

- transformar `value` e `wage` para valores numéricos

- usar as 4 colunas de informações físicas para gerar apenas um valor: `physical`

In [20]:
# --- 1. Converter 'value' e 'wage' para Números ---

def converter_valor_monetario(valor_str):
    """
    Função para converter strings como '€10.5M' ou '€200K' para um número float.
    Ex: '€10.5M' -> 10500000.0
    """
    # Verifica se o valor é uma string antes de processar
    if isinstance(valor_str, str):
        valor_str = valor_str.replace('€', '').strip()
        if 'M' in valor_str:
            return float(valor_str.replace('M', '')) * 1000000
        elif 'K' in valor_str:
            return float(valor_str.replace('K', '')) * 1000
    # Retorna o valor como está se não for uma string (pode ser NaN ou um número)
    return float(valor_str)

# Aplica a função nas colunas 'value' e 'wage' para criar colunas numéricas
df['value_eur'] = df['value'].apply(converter_valor_monetario)
df['wage_eur'] = df['wage'].apply(converter_valor_monetario)


# --- 2. Calcular a Idade a partir da Data de Nascimento ('dob') ---

# Converte a coluna 'dob' para o formato de data
df['dob'] = pd.to_datetime(df['dob'])

# Define o ano atual (considerando a data de hoje)
ano_atual = pd.to_datetime('now').year

# Calcula a idade
df['age'] = ano_atual - df['dob'].dt.year

# --- 3. Criar a Métrica de 'Fisico' ---

# O físico vai ser a média de 4 stats físicos de um jogador 

df['physical'] = ((df['stamina']  +
                df['strength']  +
                df['acceleration']  +
                df['agility'] ) / 4).round(0)

# Cria a coluna 'growth_potential' (Potencial - Overall)
df['growth_potential'] = df['potential'] - df['overall_rating']




# --- 4. Processar 'positions' para Posição Primária e Versatilidade ---

df['main_position'] = df['positions'].str.split(',').str[0].str.strip()

df['sec_positions'] = df['positions'].str.split(',').str[1:].str.join(', ').str.strip()

# Calcula a 'versatilidade' contando o número de posições que um jogador pode atuar
df['versatility'] = df['positions'].str.split(',').str.len()


cols = ["name", "age", "country_name", "main_position", "sec_positions", 
        "overall_rating", "potential", "growth_potential", 
        "value", "wage", "club_name", "club_league_name", 
        "physical", "versatility", "value_eur", "wage_eur"]

df = df[cols]


In [21]:
df.head()

,name,age,country_name,country_name,main_position,sec_positions,overall_rating,potential,growth_potential,value,wage,club_name,club_league_name,physical,versatility,value_eur,wage_eur
0,Erling Haaland,25,Norway,Norway,ST,,91,94,3,€185M,€340K,Manchester City,Premier League,82.0,1,185000000.0,340000.0
1,Kylian Mbappé,27,France,France,ST,LW,91,94,3,€181.5M,€230K,Paris Saint Germain,Ligue 1,89.0,2,181500000.0,230000.0
2,Kevin De Bruyne,34,Belgium,Belgium,CM,CAM,91,91,0,€103M,€350K,Manchester City,Premier League,78.0,2,103000000.0,350000.0
3,Rodri,29,Spain,Spain,CDM,CM,90,91,1,€122.5M,€260K,Manchester City,Premier League,74.0,2,122500000.0,260000.0
4,Harry Kane,32,England,England,ST,,90,90,0,€119.5M,€170K,FC Bayern München,Bundesliga,74.0,1,119500000.0,170000.0


Antes de prosseguir, vamos inserir os jogadores no Flamengo.

In [22]:
# Samuel Lino, Saúl, Jorginho, Emerson Royal, Danilo (Juventus), Michael, Alex Sandro (Juventus)

print(f"Número de jogadores antes da limpeza: {len(df)}")
df = df[df['club_name'] != 'Flamengo'].copy()
print(f"Número de jogadores após a limpeza: {len(df)}")

# --- 2. ADICIONAR NOVOS JOGADORES AO FLAMENGO ---

# Lista de dicionários para organizar os jogadores e seus identificadores únicos.
# Para nomes repetidos, especificamos o 'club_name' original.
jogadores_para_mudar = [
    {'name': 'Samuel Lino', 'club_name': None},
    {'name': 'Saúl', 'club_name': "Atlético Madrid"},
    {'name': 'Jorginho', 'club_name': None},
    {'name': 'Emerson Royal', 'club_name': None},
    {'name': 'Michael', 'club_name': None},
    {'name': 'Danilo', 'club_name': 'Juventus'},
    {'name': 'Alex Sandro', 'club_name': 'Juventus'}
]

NOVO_TIME = 'Flamengo'
NOVA_LIGA = 'Premier League' 

print(f"\nTransferindo jogadores para o {NOVO_TIME}...")

# Loop para Atualizar o DataFrame
for jogador in jogadores_para_mudar:
    nome_jogador = jogador['name']
    clube_original = jogador['club_name']
    
    # Cria a condição de filtro
    if clube_original:
        condicao = (df['name'] == nome_jogador) & (df['club_name'] == clube_original)
    else:
        condicao = (df['name'] == nome_jogador)
        
    # Usa .loc para encontrar as linhas que atendem à condição e alterar as colunas
    df.loc[condicao, ['club_name', 'club_league_name']] = [NOVO_TIME, NOVA_LIGA]


# --- 3. VERIFICAÇÃO FINAL ---

print("\n--- Verificação do Novo Elenco do Flamengo ---")

# Filtra e exibe todos os jogadores que agora pertencem ao Flamengo
# O resultado deve ser apenas os 7 jogadores que você adicionou
df[df['club_name'] == "Flamengo"]


Número de jogadores antes da limpeza: 18331
Número de jogadores após a limpeza: 18311

Transferindo jogadores para o Flamengo...

--- Verificação do Novo Elenco do Flamengo ---


,name,age,country_name,country_name,main_position,sec_positions,overall_rating,potential,growth_potential,value,wage,club_name,club_league_name,physical,versatility,value_eur,wage_eur
177,Jorginho,34,Italy,Italy,CDM,CM,83,83,0,€25.5M,€125K,Flamengo,Premier League,72.0,2,25500000.0,125000.0
338,Saúl,31,NaN,NaN,CM,,81,81,0,€25.5M,€65K,Flamengo,Premier League,72.0,1,25500000.0,65000.0
356,Danilo,34,NaN,NaN,CB,RB,81,81,0,€17.5M,€100K,Flamengo,Premier League,72.0,2,17500000.0,100000.0
511,Samuel Lino,26,NaN,NaN,LM,LWB,79,84,5,€26.5M,€48K,Flamengo,Premier League,78.0,2,26500000.0,48000.0
902,Emerson Royal,26,NaN,NaN,RB,CB,77,79,2,€12.5M,€56K,Flamengo,Premier League,74.0,2,12500000.0,56000.0
1095,Alex Sandro,34,NaN,NaN,CB,LB,77,77,0,€7M,€77K,Flamengo,Premier League,73.0,2,7000000.0,77000.0
1203,Michael,29,NaN,NaN,RM,LM,76,76,0,€8M,€36K,Flamengo,Premier League,78.0,2,8000000.0,36000.0


Esses serão os jogadores que serão mantidos na construção do novo elenco.

# Criando as funções de evolução dos jogadores

In [23]:
def calcular_mudanca_anual_ovr_suavizada(age, current_ovr, potential):
    """
    Versão SUAVIZADA da mudança de 'overall' base.
    Pico de carreira mais longo e declínio mais gentil.
    """
    
    if current_ovr >= potential:
        growth = 0
    else:
        # FASE DE CRESCIMENTO (antes dos ~30 anos)
        if age < 22:
            growth = np.random.uniform(1.5, 4)
        elif age < 27:
            growth = np.random.uniform(1, 3)
        elif age < 30:
            growth = np.random.uniform(0, 1)
        else:
            growth = 0

    # FASE DE DECLÍNIO (após os ~30 anos)
    if age < 30:
        decline = 0
    elif age < 33:
        decline = np.random.uniform(-1, 0)
    elif age < 36:
        decline = np.random.uniform(-2, -1)
    else:
        decline = np.random.uniform(-4, -2)

    return growth + decline


def evoluir_valor_uma_janela_v3(jogador_stats_anterior, novas_stats):
    """
    Calcula o NOVO valor de forma INCREMENTAL, baseado no valor ANTERIOR.
    """
    valor_anterior = jogador_stats_anterior['value_eur']
    
    # 1. Mudança por OVR
    ovr_change = novas_stats['overall_rating'] - jogador_stats_anterior['overall_rating']
    mult_ovr = 1.0 + (ovr_change * np.random.uniform(0.08, 0.12))
    
    # 2. Mudança por Idade (só se aplica no verão, quando a idade muda)
    mult_idade = 1.0
    age_change = novas_stats['age'] - jogador_stats_anterior['age']
    
    if age_change > 0:
        age = novas_stats['age']
        if age < 29:
            mult_idade = 1.05
        elif age < 32:
            mult_idade = 0.93
        elif age < 35:
            mult_idade = 0.88
        else:
            mult_idade = 0.82
            
    # 3. Bônus de Especulação (para potencial de crescimento)
    mult_potencial = 1.0
    if novas_stats['growth_potential'] > 0:
        mult_potencial = 1.0 + (novas_stats['growth_potential'] * 0.015) 
        
    fator_aleatorio = np.random.uniform(0.98, 1.02)
    
    novo_valor = valor_anterior * mult_ovr * mult_idade * mult_potencial * fator_aleatorio
    
    if novas_stats['overall_rating'] > 80 and novo_valor < 1000000:
        return max(novo_valor, 1000000)
        
    return round(novo_valor, -3)


def evoluir_jogador_uma_janela_v3(jogador_stats, t):
    """
    Função principal de evolução, agora usando a lógica de valor v3.
    """
    novas_stats = jogador_stats.copy()
    
    # 1. Atualiza a Idade (só no verão, t=2, t=4...)
    # (t=0 é a primeira janela, t=1 é a segunda, t=2 é a terceira)
    if t > 0 and t % 2 == 0:
        novas_stats['age'] = jogador_stats['age'] + 1
    
    # 2. Evolui Overall e Físico
    mudanca_anual_ovr = calcular_mudanca_anual_ovr_suavizada(
        novas_stats['age'], 
        novas_stats['overall_rating'], 
        novas_stats['potential']
    )
    mudanca_anual_fisico = 0
    if novas_stats['age'] > 29:
        mudanca_anual_fisico = np.random.uniform(-2, 0)
    
    fator_aleatorio_forma = np.random.normal(0, 0.5)
    
    mudanca_ovr_janela = (mudanca_anual_ovr / 2) + fator_aleatorio_forma
    mudanca_fisico_janela = mudanca_anual_fisico / 2
    
    gap_potencial = novas_stats['potential'] - novas_stats['overall_rating']
    if mudanca_ovr_janela > 0 and mudanca_ovr_janela > gap_potencial:
        mudanca_ovr_janela = max(0, gap_potencial)
    
    novas_stats['overall_rating'] = int(round(novas_stats['overall_rating'] + mudanca_ovr_janela))
    novas_stats['physical'] = int(round(novas_stats['physical'] + mudanca_fisico_janela))
    
    if novas_stats['overall_rating'] > novas_stats['potential']:
        novas_stats['overall_rating'] = novas_stats['potential']
        
    novas_stats['growth_potential'] = novas_stats['potential'] - novas_stats['overall_rating']
    
    # 4. Evolui o Valor
    novas_stats['value_eur'] = evoluir_valor_uma_janela_v3(jogador_stats, novas_stats)
    
    return novas_stats



# Início do experimento


In [24]:
TIME_ESCOLHIDO = "Flamengo"
TRANSFER_BUDGET = 150000000  # 200 Milhões de Euros
WAGE_BUDGET_YEAR = 20000000
NUM_CONTRATACOES = 23


# # Crie o mercado e o elenco atual
# mercado = df[df['club_name'] != TIME_ESCOLHIDO].copy()

# Filtra o mercado para jogadores que valem a pena
# (Ex: OVR mínimo de 70, ou valor mínimo de 1M)
mercado_completo = df[df['club_name'] != TIME_ESCOLHIDO].copy()
mercado = mercado_completo[
    (mercado_completo['overall_rating'] >= 67)
].copy()

print(f"Reduzindo o mercado de {len(mercado_completo)} para {len(mercado)} jogadores.")

elenco_atual = df[df['club_name'] == TIME_ESCOLHIDO].copy()

# AQUI ESTÁ A MUDANÇA: Calcule o tamanho final em vez de defini-lo manualmente
# TAMANHO_ELENCO_FINAL = len(elenco_atual) + NUM_CONTRATACOES
# print(f"O elenco atual do {TIME_ESCOLHIDO} tem {len(elenco_atual)} jogadores.")
# print(f"Após contratar {NUM_CONTRATACOES} jogadores, o elenco final terá {TAMANHO_ELENCO_FINAL} jogadores.")
TAMANHO_MAX_ELENCO = 30

# Requisitos mínimos de posições para o elenco final
# requisitos_posicao = {
#     'GK': 3,
#     'CB': 4,
#     'LB': 2,
#     'RB': 2,
#     'CDM': 3,
#     'CM': 4,
#     'LW': 2,
#     'RW': 2,
#     'ST': 3
# }

requisitos_posicao = {
    'GK': 3,
    'CB': 4,
    'LFB': 2,  # Corrigido de 'FB': 4
    'RFB': 2,  # Corrigido de 'FB': 4
    'MC': 7,
    'LWG': 2,  # Corrigido de 'WG': 4
    'RWG': 2,  # Corrigido de 'WG': 4
    'ST': 3
}

# Definição dos mapas de posição (para agregar 'CM', 'CDM', 'CAM', etc.)
formacao_titular = {
    'GK': 1,
    'CB': 2,
    'LFB': 1,  # Full-back direito
    'RFB': 1,  # Full-back direito
    'MC': 3,  # Meio-Campista (Geral)
    'LWG': 1,  # Winger esquerdo
    'RWG': 1,
    'ST': 1
}

mapa_posicoes = { # Essa função faz um mapeamento de posições para a montagem da escalação
    'GK': 'GK',
    'CB': 'CB',
    'LB': 'LFB',
    'RB': 'RFB',
    'LWB': 'LFB',
    'RWB': 'RFB',
    'CDM': 'MC',
    'CM': 'MC',
    'CAM': 'MC',
    'LW': 'LWG',
    'RW': 'RWG',
    'LM': 'LWG',
    'RM': 'RWG',
    'ST': 'ST',
    'CF': 'ST'
}

NUM_TITULARES = 11

# É importante diferenciar titulares de reservas tanto para a química, quanto para garantir que o modelo
# monte um elenco equilibrado. Sem isso, ele pode preferir contratar 3 goleiros top-class e negligenciar
# o resto do time, por exemplo.

Reduzindo o mercado de 18304 para 8447 jogadores.


In [25]:
# Dicionários de dados separados
elenco_data = elenco_atual.to_dict('index')
players_data = mercado.to_dict('index') # 'players_data' é só o mercado

# IDs
elenco_ids = elenco_atual.index.tolist()
player_ids = mercado.index.tolist() # IDs do mercado
elenco_final_ids = elenco_ids + player_ids # IDs de todos os jogadores (atuais + mercado)

# Dicionário de dados COMPLETO (necessário para 'titular_vars')
players_data_completo = {}
players_data_completo.update(elenco_data)
players_data_completo.update(players_data)


/var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/ipykernel_43436/2719003106.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  elenco_data = elenco_atual.to_dict('index')
/var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/ipykernel_43436/2719003106.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  players_data = mercado.to_dict('index') # 'players_data' é só o mercado


In [26]:
print("Iniciando o pré-cálculo da evolução dos jogadores...")
start_time = time.time()

# Define o número de janelas que queremos simular (t=0, t=1, t=2, t=3)
NUM_JANELAS = 4

# O "almanaque" final
dados_temporais = {}

# 'players_data_completo' é o dicionário que você já tem, com os dados da janela t=0
for player_id, stats_t0 in players_data_completo.items():
    
    # Inicializa o dicionário para este jogador
    dados_temporais[player_id] = {}
    
    # Adiciona os dados da primeira janela (t=0)
    dados_temporais[player_id][0] = stats_t0
    
    # Loop para simular as janelas futuras (t=1, t=2, t=3)
    for t in range(1, NUM_JANELAS):
        
        # Pega os stats da janela anterior
        stats_anterior = dados_temporais[player_id][t-1]
        
        # Simula a evolução para a janela atual 't'
        try:
            novas_stats = evoluir_jogador_uma_janela_v3(stats_anterior, t)
        except Exception as e:
            # Captura erros (ex: dados faltando) para um jogador específico
            # e continua o processo para os outros
            print(f"Erro ao processar jogador {stats_anterior.get('name', player_id)}: {e}")
            novas_stats = stats_anterior # Mantém os stats antigos se a evolução falhar
            
        # Armazena os novos stats projetados
        dados_temporais[player_id][t] = novas_stats

end_time = time.time()
print(f"Pré-cálculo concluído em {end_time - start_time:.2f} segundos.")
print(f"Total de {len(dados_temporais)} jogadores processados para {NUM_JANELAS} janelas.")


# --- Verificação da Estrutura de Dados ---
print("\n--- Verificando a evolução de um jogador (ex: Samuel Lino) ---")

# Pega o ID do Samuel Lino (do seu elenco atual)
# Nota: Você precisará ajustar o 'nome' se ele não for único
try:
    SAMUEL_LINO_ID = [pid for pid, data in elenco_data.items() if data['name'] == 'Samuel Lino'][0]

    for t in range(NUM_JANELAS):
        stats = dados_temporais[SAMUEL_LINO_ID][t]
        print(f"Janela {t} (Idade {stats['age']}): "
              f"OVR: {stats['overall_rating']}, "
              f"Pot: {stats['potential']}, "
              f"Valor: €{stats['value_eur']:,}")
except IndexError:
    print("Não foi possível encontrar 'Samuel Lino' no elenco_data para o exemplo.")
    # Apenas pega o primeiro jogador do dataset para o exemplo
    primeiro_id = list(dados_temporais.keys())[0]
    stats_exemplo = dados_temporais[primeiro_id]
    print(f"\nExibindo jogador aleatório (ID: {primeiro_id}):")
    for t in range(NUM_JANELAS):
        stats = stats_exemplo[t]
        print(f"Janela {t}: OVR: {stats['overall_rating']}, Valor: €{stats['value_eur']:,}")

Iniciando o pré-cálculo da evolução dos jogadores...
Pré-cálculo concluído em 0.15 segundos.
Total de 8454 jogadores processados para 4 janelas.

--- Verificando a evolução de um jogador (ex: Samuel Lino) ---
Janela 0 (Idade 26): OVR: 79, Pot: 84, Valor: €26,500,000.0
Janela 1 (Idade 26): OVR: 80, Pot: 84, Valor: €30,649,000.0
Janela 2 (Idade 27): OVR: 80, Pot: 84, Valor: €34,514,000.0
Janela 3 (Idade 27): OVR: 80, Pot: 84, Valor: €36,280,000.0


## Definição do modelo (v3 - com dados temporais)

In [27]:
import pulp

# --- 1. Definições Iniciais ---

# Define os períodos de tempo (4 janelas = 2 anos)
JANELAS = list(range(NUM_JANELAS)) # Ex: [0, 1, 2, 3]

# IDs dos jogadores (como antes)
elenco_ids = elenco_atual.index.tolist()
mercado_ids = mercado.index.tolist()
todos_os_ids = elenco_ids + mercado_ids # Todos os jogadores no nosso universo

# Orçamento inicial (o orçamento para t>0 será uma variável)
ORCAMENTO_INICIAL = TRANSFER_BUDGET # O orçamento que você tem agora (t=0)
ORCAMENTO_SALARIAL_INICIAL = WAGE_BUDGET_YEAR # O teto salarial

# --- 2. Inicialização do Modelo ---
model = pulp.LpProblem("Planejamento_Estrategico_Elenco", pulp.LpMaximize)

# --- 3. Novas Variáveis de Decisão (Indexadas pelo Tempo) ---

# IDs para as variáveis (jogador, janela)
var_indices = [(i, t) for i in todos_os_ids for t in JANELAS]
# IDs apenas para o mercado (comprar)
var_indices_mercado = [(j, t) for j in mercado_ids for t in JANELAS]

# 'no_elenco_vars': 1 se o jogador 'i' ESTÁ no nosso elenco na janela 't'
no_elenco_vars = pulp.LpVariable.dicts("NoElenco", var_indices, cat='Binary')

# 'contratar_vars': 1 se o jogador 'j' (do mercado) é CONTRATADO na janela 't'
contratar_vars = pulp.LpVariable.dicts("Contratar", var_indices_mercado, cat='Binary')

# 'vender_vars': 1 se o jogador 'i' (do nosso elenco) é VENDIDO na janela 't'
vender_vars = pulp.LpVariable.dicts("Vender", var_indices, cat='Binary')

# 'titular_vars': 1 se o jogador 'i' é TITULAR na temporada da janela 't'
titular_vars = pulp.LpVariable.dicts("Titular", var_indices, cat='Binary')

# Variáveis contínuas para o orçamento
orcamento_transfer = pulp.LpVariable.dicts("OrcamentoTransfer", JANELAS, lowBound=0)
orcamento_salario_disponivel = pulp.LpVariable.dicts("OrcamentoSalarial", JANELAS, lowBound=0)


# --- 4. Restrições (O Coração do Modelo Dinâmico) ---

# A. Restrições Iniciais (t=0)
# Define o estado inicial do nosso elenco
for i in todos_os_ids:
    if i in elenco_ids:
        # Jogadores do elenco atual COMEÇAM no elenco
        model += no_elenco_vars[i, 0] == 1, f"Inicio_Elenco_{i}"
    else:
        # Jogadores do mercado NÃO COMEÇAM no elenco
        model += no_elenco_vars[i, 0] == 0, f"Inicio_Mercado_{i}"

# Define os orçamentos iniciais
model += orcamento_transfer[0] == ORCAMENTO_INICIAL, "Orcamento_Inicial_Transfer"
model += orcamento_salario_disponivel[0] == ORCAMENTO_SALARIAL_INICIAL, "Orcamento_Inicial_Salario"


# B/C. Restrições de Fluxo e Operação (Conectando as Janelas)
for t in JANELAS:
    
    # --- Restrições que se aplicam a t > 0 ---
    # (Estas restrições definem a transição E a validade dos novos elencos)
    if t > 0: 
        
        # B.1. Fluxo de Jogadores (A "Memória" do Elenco)
        for i in todos_os_ids:
            if i in mercado_ids:
                 model += no_elenco_vars[i, t] == no_elenco_vars[i, t-1] - vender_vars[i, t-1] + contratar_vars[i, t-1], f"Fluxo_Elenco_Mercado_{i}_{t}"
            else: # Jogadores do elenco original (não podem ser re-contratados)
                 model += no_elenco_vars[i, t] == no_elenco_vars[i, t-1] - vender_vars[i, t-1], f"Fluxo_Elenco_Original_{i}_{t}"

        # B.2. Fluxo de Orçamento
        gastos_transfer_t_menos_1 = pulp.lpSum([dados_temporais[j][t-1]['value_eur'] * contratar_vars[j, t-1] for j in mercado_ids])
        receitas_vendas_t_menos_1 = pulp.lpSum([dados_temporais[i][t-1]['value_eur'] * vender_vars[i, t-1] for i in todos_os_ids])
        model += orcamento_transfer[t] == orcamento_transfer[t-1] - gastos_transfer_t_menos_1 + receitas_vendas_t_menos_1, f"Fluxo_Orcamento_Transfer_{t}"

        # C.1. Teto Salarial Total (Aplicado a t > 0)
        salario_total_t = pulp.lpSum([dados_temporais[i][t]['wage_eur']*52 * no_elenco_vars[i, t] for i in todos_os_ids])
        model += salario_total_t <= ORCAMENTO_SALARIAL_INICIAL, f"Teto_Salarial_{t}"
        
        # C.5. Restrições de Formação Titular (Aplicado a t > 0)
        for pos_formacao, num_necessarios in formacao_titular.items():
            jogadores_da_posicao = [
                i for i in todos_os_ids 
                if mapa_posicoes.get(dados_temporais[i][t]['main_position']) == pos_formacao
            ]
            model += pulp.lpSum([titular_vars[i, t] for i in jogadores_da_posicao]) == num_necessarios, f'Titulares_{pos_formacao}_{t}'
            
        # C.6. Restrições de Profundidade do Elenco (Aplicado a t > 0)
        for pos_agregada, min_req in requisitos_posicao.items():
            jogadores_da_posicao = [
                i for i in todos_os_ids 
                if mapa_posicoes.get(dados_temporais[i][t]['main_position']) == pos_agregada
            ]
            model += pulp.lpSum([no_elenco_vars[i, t] for i in jogadores_da_posicao]) \
                         >= min_req, f"Restricao_Min_{pos_agregada}_{t}"

        # Tamanho Máximo do Elenco (Aplicado a t > 0)
        model += pulp.lpSum([no_elenco_vars[i, t] for i in todos_os_ids]) <= TAMANHO_MAX_ELENCO, f"Tamanho_Max_Elenco_{t}"

    
    # --- Restrições que se aplicam a TODAS as janelas (t=0, 1, 2, 3) ---
    
    # C.2. Um jogador não pode ser vendido se não está no elenco
    # C.3. Um jogador não pode ser titular se não está no elenco
    for i in todos_os_ids:
        model += vender_vars[i, t] <= no_elenco_vars[i, t], f"Ligacao_Venda_{i}_{t}"
        model += titular_vars[i, t] <= no_elenco_vars[i, t], f"Ligacao_Titular_{i}_{t}"
    
    # C.4. Um jogador só pode ser comprado UMA VEZ (Restrição Global)
    if t == 0:
        for j in mercado_ids:
            # A soma de contratações de um jogador ao longo de TODAS as janelas deve ser no máximo 1
             model += pulp.lpSum([contratar_vars[j, t_futuro] for t_futuro in JANELAS]) <= 1, f"Compra_Unica_{j}"


# --- 5. Definição da Função Objetivo (Acumulada) ---

# Pesos
PESO_TITULAR = 1.0
PESO_RESERVA = 0.15
w_qualidade = 0.5
w_potencial = 0.3
w_fisico = 0.2

obj_total = [] # Lista para guardar o score de cada janela

for t in JANELAS:
    
    # A CORREÇÃO: Não calculamos o score do elenco inicial (t=0),
    # pois ele é fixo e não pode ser otimizado.
    if t > 0:
        score_reserva_t = {}
        score_titular_bonus_t = {}
        
        for i in todos_os_ids:
            p_data = dados_temporais[i][t] # Pega os stats projetados para a janela 't'
            
            score_base = (p_data['overall_rating'] * w_qualidade +
                          p_data.get('growth_potential', 0) * w_potencial +
                          p_data['physical'] * w_fisico)
            
            score_reserva_t[i] = score_base * PESO_RESERVA
            score_titular_bonus_t[i] = score_base * (PESO_TITULAR - PESO_RESERVA)

        # O valor do elenco na janela 't'
        obj_elenco_t = pulp.lpSum([score_reserva_t[i] * no_elenco_vars[i, t] for i in todos_os_ids])
        
        # O bônus dos titulares na janela 't'
        obj_bonus_titulares_t = pulp.lpSum([score_titular_bonus_t[i] * titular_vars[i, t] for i in todos_os_ids])
        
        # O score total da janela 't'
        obj_total.append(obj_elenco_t + obj_bonus_titulares_t)

# A função objetivo final é a SOMA dos scores de todas as janelas (exceto t=0)
model += pulp.lpSum(obj_total), "Score_Acumulado_Total"

In [28]:
# --- 6. Resolver o Modelo Dinâmico ---

print("Iniciando a resolução do modelo dinâmico...")
print("Isso pode levar alguns minutos...")
start_time = time.time()

# Chama o solver
# model.solve()
# Define um "gap" de otimalidade relativo de 0.5% (0.005)
# O solver vai parar assim que encontrar uma solução que esteja
# a 0.5% da melhor solução teórica possível.
model.solve(pulp.PULP_CBC_CMD(gapRel=0.005))

end_time = time.time()
print(f"Resolução concluída em {end_time - start_time:.2f} segundos.")


# --- 7. Exibir os Resultados (Função de Análise Dinâmica) ---

print(f"\nStatus da Solução: {pulp.LpStatus[model.status]}\n")

# Verifica se uma solução ótima foi encontrada
if pulp.LpStatus[model.status] == 'Optimal':
    
    print("--- PLANO ESTRATÉGICO DE 2 ANOS (4 JANELAS) ---")
    
    # Itera por CADA JANELA de tempo para mostrar as decisões
    for t in JANELAS:
        print("\n" + "="*30)
        print(f"         JANELA {t} (Verão {2024 + t//2} / Inverno {2025 + t//2})")
        print("="*30)
        
        # --- A. Decisões de Transferência (O que aconteceu nesta janela) ---
        print("\n  A. Decisões de Transferência:")
        
        contratados_t = []
        for j in mercado_ids:
            if contratar_vars[j, t].varValue == 1:
                stats = dados_temporais[j][t]
                contratados_t.append(stats)
                print(f"    [CONTRATAÇÃO] {stats['name']} (OVR: {stats['overall_rating']}) "
                      f"por €{stats['value_eur']:,.0f}")

        vendidos_t = []
        for i in todos_os_ids: # Itera sobre todos, pois podemos vender um recém-contratado
            if vender_vars[i, t].varValue == 1:
                stats = dados_temporais[i][t]
                vendidos_t.append(stats)
                print(f"    [VENDA]         {stats['name']} (OVR: {stats['overall_rating']}) "
                      f"por €{stats['value_eur']:,.0f}")
        
        if not contratados_t and not vendidos_t:
            print("    Nenhuma transação nesta janela.")

        # --- B. Time Titular (O time para esta temporada) ---
        print("\n  B. Time Titular da Temporada:")
        
        time_titular_t = []
        for i in todos_os_ids:
            if titular_vars[i, t].varValue == 1:
                time_titular_t.append(dados_temporais[i][t])
        
        # Ordena o time titular por posição
        mapa_ordem = {pos: idx for idx, pos in enumerate(formacao_titular.keys())}
        time_titular_ordenado_t = sorted(
            time_titular_t, 
            key=lambda x: mapa_ordem.get(mapa_posicoes.get(x['main_position']), 99)
        )
        
        for jogador in time_titular_ordenado_t:
            print(f"    - {jogador['main_position']:<4} | {jogador['name']:<25} | "
                  f"(Idade: {jogador['age']}, OVR: {jogador['overall_rating']})")

        # --- C. Status do Elenco e Finanças (No final desta janela) ---
        print("\n  C. Status do Elenco e Finanças (Final da Janela):")
        
        elenco_total_t = []
        salario_total_t = 0
        for i in todos_os_ids:
            if no_elenco_vars[i, t].varValue == 1:
                stats = dados_temporais[i][t]
                elenco_total_t.append(stats)
                salario_total_t += stats['wage_eur'] * 52
                
        print(f"    - Tamanho do Elenco: {len(elenco_total_t)} jogadores")
        
        # Exibe o status do orçamento
        # (O orçamento só é atualizado na próxima janela, t+1)
        if t + 1 < NUM_JANELAS:
            print(f"    - Salário Anual: €{salario_total_t:,.0f} "
                  f"(Teto: €{ORCAMENTO_SALARIAL_INICIAL:,.0f})")
            print(f"    - Orçamento de Transferência p/ Próxima Janela (t={t+1}): "
                  f"€{orcamento_transfer[t+1].varValue:,.0f}")
        else:
            print(f"    - Salário Anual Final: €{salario_total_t:,.0f}")
            print(f"    - Orçamento Final (t={t}): €{orcamento_transfer[t].varValue:,.0f}")


else:
    print("O modelo não encontrou uma solução ótima (Pode ser Infactível ou Ilimitado).")
    print("\n--- DICAS PARA DEBUGAR (INFEASIBLE) ---")
    print("1. O Orçamento está muito baixo? (Tente aumentar ORCAMENTO_INICIAL)")
    print("2. O Teto Salarial está muito baixo? (Tente aumentar ORCAMENTO_SALARIAL_INICIAL)")
    print("3. As Restrições de Posição são muito rígidas? (Tente diminuir 'requisitos_posicao')")
    print("4. O Mercado tem jogadores suficientes para todas as posições em todas as janelas?")

Iniciando a resolução do modelo dinâmico...
Isso pode levar alguns minutos...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/danielbarros/projects/Otimizacao_Trab/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/a9b40282ee1b4f528508d3e51fff782e-pulp.mps -max -ratio 0.005 -timeMode elapsed -branch -printingOptions all -solution /var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/a9b40282ee1b4f528508d3e51fff782e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 109959 COLUMNS
At line 861642 RHS
At line 971597 BOUNDS
At line 1106834 ENDATA
Problem MODEL has 109954 rows, 135241 columns and 430486 elements
Coin0008I MODEL read with 0 errors
ratioGap was changed from 0 to 0.005
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2363.87 - 4.31 seconds
Cgl0002I 25341 variables fixed
Cgl0003I 0 fixed, 0 tightened bou